### Part 1: data pre-processing

In [1]:
# Python code to build Machine Learning model for hurricane intensity forecast  
import pandas as pd # For data manipulation and analysis
pd.set_option('display.max_columns', 500)
import numpy as np # For scientific computing

#!pip3 install sklearn # Install machine learning library
import sklearn # For machine learning library
from sklearn.ensemble import RandomForestClassifier  # Random forest classifier
#from sklearn.ensemble import ExtraTreesClassifier    # Extra tree classifier
from sklearn.metrics import confusion_matrix # Compute confusion matrix to evaluate the accuracy of a classification.
from sklearn.metrics import brier_score_loss  # Compute the Brier score

import matplotlib.pyplot as plt  #plotting library
from global_land_mask import globe # a library to check whether a lat&lon on sea or land

##### 1.1 Eco Loss data

In [ ]:
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/Hurricane_Ecoloss_rawdata_Wanying.csv'

# Read EcoLoss data
Ecoloss = pd.read_csv(fname)
Ecoloss.head()

In [ ]:
# Extract month from landfall date
Ecoloss['MONTH'] = Ecoloss['landfall date'].apply(lambda x: str(x)[0:3])

# Extract date from landfall date
Ecoloss['DATE'] = Ecoloss['landfall date'].apply(lambda x: str(x)[4:6])

# Extract year from landfall date
Ecoloss['YEAR'] = Ecoloss['landfall date'].apply(lambda x: str(x)[7:11])

Ecoloss.sort_values("YEAR",inplace=True)

Ecoloss.head()

In [ ]:
Month2Int={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
Ecoloss['MONTHINT']=Ecoloss['MONTH'].apply(lambda x: Month2Int[x])
Ecoloss.head()

In [ ]:
Ecoloss=Ecoloss[(Ecoloss['YEAR']>='1982') & (Ecoloss['YEAR']<='2017')]
Ecoloss.head()

In [ ]:
Ecoloss.to_csv('C:\\Users\\HP\\Desktop\\Ecoloss_from1982.csv')

In [ ]:
Ecoloss.YEAR.value_counts().sort_index()

In [ ]:
Ecoloss=Ecoloss.reset_index(drop=True)

In [ ]:
# set a 'key' to match ships dataset
# upper Name[0:4]+Year+Month(int)
Ecoloss['KEY']=0
for i in range(0,Ecoloss.shape[0]):
    Ecoloss.loc[i,'KEY']='-'.join([Ecoloss.loc[i,'storm name'][0:4].upper(),Ecoloss.loc[i,'YEAR'],Ecoloss.loc[i,'MONTHINT']])
Ecoloss.head()

In [ ]:
Ecoloss.shape[0]

##### 1.2 Ships data

In [ ]:
#====================================
#Read SHIPS spread-sheet data
#====================================
# Set up the location of the SHIPS data
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/SHIPS_RII_fcst_ATL_Alicia_from1982%20_Wanying.csv'
#fname='Dataset_SHIPS_RII_EPAC.csv'

# Read SHIPS data
ships = pd.read_csv(fname)
ships

In [ ]:
#================================
# Set up parameters
#================================
# Year range for training and validating
#year_train=['1998','2008']

# Year range for forecast
#year_fcst=['2009','2014']

# Variable names for predictors
#PredictorName=['PER','SHRD','D200','TPW','PC2','SDBT','POT','OHC','VMX0']

# Variable name for predictand
TargetName='DELV24'

# Threshold of Rapid Intensification 
RIValue=30

# Climatology of RI (30 kt) frequency at Atlantic basin (Kaplan et al. 2015)
clim=0.125   #ATL 30 kt
#clim=0.084   #EPAC 30 kt

In [ ]:
#================================
# Data pre-processing
#================================
# Set all 9999s as NaNs
ships = ships.replace(9999,np.NaN)

# drop NaNs
ships=ships.dropna()

# Pad the date columns with 00 for the year 2000
ships['DATE'] = ships['DATE'].apply(lambda x: str(x).zfill(6))

# Extract month from date
ships['MONTH'] = ships['DATE'].apply(lambda x: str(x)[2:4])

# Extract year from date
ships['YEAR'] = ships['DATE'].apply(lambda x: ('19' + str(x)[0:2]) if (str(x)[0:1]!= '0' and str(x)[0:1]!= '1') else ('20' + str(x)[0:2]))

# Extract day from date
ships['DAY'] = ships['DATE'].apply(lambda x: str(x)[4:6])

# Set the target column
ships['TAR'] = ships[TargetName].apply(lambda x: 1 if x >= RIValue else 0)
ships.head()

In [ ]:
ships=ships.reset_index(drop=True)
# set a 'key' to match ships dataset
# upper Name[0:4]+Year+Month(int)
ships['KEY']=0
for i in range(0,ships.shape[0]):
    ships.loc[i,'KEY']='-'.join([ships.loc[i,'NAME'],ships.loc[i,'YEAR'],ships.loc[i,'MONTH']])
ships.head()

In [ ]:
ships.shape[0]

In [ ]:
ships.YEAR.value_counts().sort_index()

In [ ]:
# combine two tables
Merge = pd.merge(ships, Ecoloss, on=['KEY'])
Merge

In [ ]:
Merge1=Merge.copy()
Merge1['intDay']=Merge1['DAY'].apply(lambda x: int(x))
Merge1['intDate']=Merge1['DATE_y'].apply(lambda x: int(x))
Merge2=Merge1[Merge1['intDay']==Merge1['intDate']]
Merge2.head()

In [ ]:
Merge2.YEAR_y.value_counts().sort_index()

##### 1.3 population dataset

In [ ]:
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/population_stateLevel_Wanying.csv'

# Read Population data
population = pd.read_csv(fname)
population.head()

In [ ]:
population.shape[0]

In [ ]:
# row to col
State=population['State']
Year=range(1998,2020)
popu=pd.DataFrame(columns=('State', 'Year', 'population'))
for i in range(0,52):
    stateCur=State[i]
    for j in range(0,22):
        yearCur=str(Year[j])
        col='y'+yearCur
        popu = popu.append([{'State':stateCur,'Year':yearCur,'population':population.loc[i,col]}], ignore_index=True)
popu.head()

##### 1.4 GDP

In [ ]:
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/GDP_stateLevel_Wanying.csv'

# Read GDP data
GDP = pd.read_csv(fname)
GDP.head()

In [ ]:
GDP.shape[0]

In [ ]:
# row to col
State=GDP['State']
Year=range(1998,2020)
gdp=pd.DataFrame(columns=('State', 'Year', 'GDP'))
for i in range(0,51):
    stateCur=State[i]
    for j in range(0,22):
        yearCur=str(Year[j])
        col='y'+yearCur
        gdp = gdp.append([{'State':stateCur,'Year':yearCur,'GDP':GDP.loc[i,col]}], ignore_index=True)
gdp.head()

### 1.5 Merge

In [ ]:
Merge2=Merge2.rename(columns={'landfall state':'State','YEAR_y':'Year'})

In [ ]:
# combine two tables
Merge3 = pd.merge(Merge2,gdp,how='left',on=['State','Year'])
Merge3.head()

In [ ]:
# combine two tables
Merge4 = pd.merge(Merge3,popu,how='left',on=['State','Year'])
Merge4

### 1.6 Match grid-GDP(lat&lon level GDP) dataset

##### 1.6.1 read data

In [ ]:
# use grid econ data created by Yale University
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/USA_econData_latLonLevel_Wanying.csv'

# Read EcoLoss data
gridGDP = pd.read_csv(fname)
gridGDP

##### 1.6.2 Match Lon,Lat and Year

In [ ]:
# round Lon and Lat in 'Merge4'
Merge4['LatInt']=Merge4['LAT'].apply(lambda x:round(x))
Merge4['LonInt']=Merge4['LON'].apply(lambda x:round(x))
Merge4

In [ ]:
# match LON and LAT
for i in range(len(Merge4)):
    # Lat
    x0=Merge4.loc[i,'LAT']
    x1=Merge4.loc[i,'LatInt']
    x2=x1-1
    x3=x1+1
    x=[x1,x2,x3]
    # lon all are negative values
    y0=Merge4.loc[i,'LON']
    y1seires=gridGDP['longitude'][gridGDP['lat']==x1]
    y1=y1seires[np.argmin(abs(y1seires-y0))] if len(y1seires)!=0 else [0]
    y2seires=gridGDP['longitude'][gridGDP['lat']==x2]
    y2=y2seires[np.argmin(abs(y2seires-y0))] if len(y2seires)!=0 else [0]
    y3seires=gridGDP['longitude'][gridGDP['lat']==x3]
    y3=y3seires[np.argmin(abs(y3seires-y0))] if len(y3seires)!=0 else [0]
    y=[y1,y2,y3]
    import math
    dist=[0,0,0]
    for j in range(0,3):
        dist[j]=math.sqrt((x[j]-x0)**2+(y[j]-y0)**2)
    index_min=np.argmin(dist)
    #print(i,x[index_min],y[index_min])
    Merge4.loc[i,'grid_Lat']=x[index_min]
    Merge4.loc[i,'grid_Lon']=y[index_min]
Merge4

In [ ]:
Merge4['Year']=Merge4['Year'].apply(lambda x:int(x))

In [ ]:
# match years
y90=[1988,1989,1990,1991,1992]
y95=[1993,1994,1995,1996,1997]
y00=[1998,1999,2000,2001,2002]
y05=[2003,2004,2005,2006,2007]
for i in range(len(Merge4)):
    if Merge4.loc[i,'Year'] in y90 or Merge4.loc[i,'Year']<=1988:
        grid_year='90'
        Merge4.loc[i,'GDPMER']=gridGDP['GDPMER90'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
        Merge4.loc[i,'popgpw']=gridGDP['popgpw_1990_27_rswb'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
    elif Merge4.loc[i,'Year'] in y95:
        grid_year='95'
        Merge4.loc[i,'GDPMER']=gridGDP['GDPMER95'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
        Merge4.loc[i,'popgpw']=gridGDP['popgpw_1995_27_rswb'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
    elif Merge4.loc[i,'Year'] in y00:
        grid_year='00'
        Merge4.loc[i,'GDPMER']=gridGDP['GDPMER00'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
        Merge4.loc[i,'popgpw']=gridGDP['popgpw_2000_27_rswb'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
    else:
        grid_year='05'
        Merge4.loc[i,'GDPMER']=gridGDP['GDPMER05'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
        Merge4.loc[i,'popgpw']=gridGDP['popgpw_2005_211_rswb'][(gridGDP['lat']==Merge4.loc[i,'grid_Lat']) & (gridGDP['longitude']==Merge4.loc[i,'grid_Lon'])].values[0]
Merge4

In [ ]:
# delete duplicate rows
Merge4=Merge4.drop_duplicates(subset =['NAME','DATE_x','HOUR'] , keep = 'first')
Merge4

In [ ]:
# output data
Merge5=Merge4.copy()
Merge5.to_csv('C:\\Users\\HP\\Desktop\\Merge5_from1982.csv',index= False)

##### 1.6.3 input new 'Merge' dataset from local path directly

In [2]:
# input new 'Merge' dataset
Merge5 = pd.read_csv('C:\\Users\\HP\\Desktop\\Merge5_from1982.csv')

In [3]:
Merge5

,NAME,DATE_x,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST,MONTH_x,YEAR_x,DAY,TAR,KEY,storm name,landfall date,damage ranke,current damage($ 2020),base damage,State,category,winds(MPH),MONTH_y,DATE_y,Year,MONTHINT,intDay,intDate,GDP,population,LatInt,LonInt,grid_Lat,grid_Lon,GDPMER,popgpw
0,ALIC,830818,0,95,28.4,-94.8,969.0,AL031983,-15.0,-60.0,-70.0,-75.0,20.0,2.0,15,60,58.0,11.2,-46,0,83,13.0,5.8,14.3,23.0,-3.1,7.0,5.5,60.5,29.8,8,1983,18,0,ALIC-1983-08,Alicia,"Aug 18,1983",26,2.233000e+10,3.000000e+09,TX,3,115,Aug,18,1983,8,18,18,NaN,NaN,28,-95,29.0,-95.0,6.817196,2.341434e+05
1,DIAN,840913,0,85,33.8,-77.4,972.0,AL101984,-20.0,-40.0,-40.0,-30.0,-10.0,1.7,-41,53,87.0,10.6,-37,0,465,-23.0,5.4,11.0,-61.0,1.0,0.9,4.0,58.2,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-77,34.0,-77.0,1.767393,7.252184e+04
2,DIAN,840913,12,65,34.0,-78.3,990.0,AL101984,-20.0,-20.0,-10.0,-5.0,-20.0,5.2,-21,53,60.0,12.9,-41,0,741,-15.0,8.3,12.0,-44.0,-0.1,0.0,7.1,56.4,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-78,34.0,-78.0,7.728861,3.242476e+05
3,ELEN,850902,0,110,29.4,-85.9,953.0,AL051985,-10.0,-65.0,-85.0,-90.0,5.0,9.9,13,61,76.0,12.4,-45,0,212,-21.0,9.9,17.9,12.0,-5.5,3.4,3.3,61.1,29.4,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,29,-86,29.0,-86.0,0.239075,1.364132e+04
4,ELEN,850902,12,100,30.2,-88.8,959.0,AL051985,-55.0,-75.0,-80.0,-80.0,-10.0,11.9,2,59,75.0,12.0,-59,0,324,-19.0,13.1,22.1,-7.0,-5.0,0.1,4.7,61.5,29.3,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,30,-89,30.0,-89.0,11.205411,5.601347e+05
5,KATE,851121,0,105,26.8,-86.5,954.0,AL131985,-10.0,-25.0,-55.0,-65.0,0.0,18.7,66,75,92.0,10.0,15,0,0,-101.0,22.8,31.6,60.0,-1.3,0.0,5.2,58.1,25.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,27,-86,27.0,-83.0,44.292372,1.653082e+06
6,KATE,851121,6,100,27.5,-86.6,961.0,AL131985,-15.0,-35.0,-55.0,-65.0,-5.0,12.8,110,69,88.0,12.9,17,0,236,-94.0,23.6,34.8,108.0,3.0,0.0,10.8,56.3,25.3,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-87,29.0,-86.0,0.239075,1.364132e+04
7,KATE,851121,12,95,28.3,-86.5,965.0,AL131985,-15.0,-45.0,-55.0,-60.0,-10.0,15.9,93,69,91.0,12.1,18,0,553,-100.0,24.6,37.1,97.0,9.3,0.0,14.6,55.8,24.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-86,29.0,-86.0,0.239075,1.364132e+04
8,BONN,860626,0,70,28.2,-92.9,999.0,AL021986,-5.0,-40.0,-50.0,-55.0,15.0,7.6,-16,67,45.0,17.0,-68,0,190,52.0,12.8,15.5,15.0,0.7,5.7,3.6,56.1,28.7,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,28,-93,29.0,-93.0,0.724109,3.454524e+04
9,BONN,860626,6,75,29.0,-93.7,995.0,AL021986,-40.0,-50.0,-55.0,-65.0,10.0,15.2,-36,65,39.0,19.0,-71,0,250,54.0,20.1,23.5,-17.0,1.5,3.8,3.2,55.1,28.6,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,29,-94,29.0,-94.0,2.691408,1.015603e+05


### 1.7 check lat&lon points are on sea/land

In [4]:
# True: on land, False: in sea
Merge5['is_land']=globe.is_land(Merge5['LAT'], Merge5['LON'])
Merge5

,NAME,DATE_x,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST,MONTH_x,YEAR_x,DAY,TAR,KEY,storm name,landfall date,damage ranke,current damage($ 2020),base damage,State,category,winds(MPH),MONTH_y,DATE_y,Year,MONTHINT,intDay,intDate,GDP,population,LatInt,LonInt,grid_Lat,grid_Lon,GDPMER,popgpw,is_land
0,ALIC,830818,0,95,28.4,-94.8,969.0,AL031983,-15.0,-60.0,-70.0,-75.0,20.0,2.0,15,60,58.0,11.2,-46,0,83,13.0,5.8,14.3,23.0,-3.1,7.0,5.5,60.5,29.8,8,1983,18,0,ALIC-1983-08,Alicia,"Aug 18,1983",26,2.233000e+10,3.000000e+09,TX,3,115,Aug,18,1983,8,18,18,NaN,NaN,28,-95,29.0,-95.0,6.817196,2.341434e+05,False
1,DIAN,840913,0,85,33.8,-77.4,972.0,AL101984,-20.0,-40.0,-40.0,-30.0,-10.0,1.7,-41,53,87.0,10.6,-37,0,465,-23.0,5.4,11.0,-61.0,1.0,0.9,4.0,58.2,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-77,34.0,-77.0,1.767393,7.252184e+04,False
2,DIAN,840913,12,65,34.0,-78.3,990.0,AL101984,-20.0,-20.0,-10.0,-5.0,-20.0,5.2,-21,53,60.0,12.9,-41,0,741,-15.0,8.3,12.0,-44.0,-0.1,0.0,7.1,56.4,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-78,34.0,-78.0,7.728861,3.242476e+05,True
3,ELEN,850902,0,110,29.4,-85.9,953.0,AL051985,-10.0,-65.0,-85.0,-90.0,5.0,9.9,13,61,76.0,12.4,-45,0,212,-21.0,9.9,17.9,12.0,-5.5,3.4,3.3,61.1,29.4,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,29,-86,29.0,-86.0,0.239075,1.364132e+04,False
4,ELEN,850902,12,100,30.2,-88.8,959.0,AL051985,-55.0,-75.0,-80.0,-80.0,-10.0,11.9,2,59,75.0,12.0,-59,0,324,-19.0,13.1,22.1,-7.0,-5.0,0.1,4.7,61.5,29.3,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,30,-89,30.0,-89.0,11.205411,5.601347e+05,False
5,KATE,851121,0,105,26.8,-86.5,954.0,AL131985,-10.0,-25.0,-55.0,-65.0,0.0,18.7,66,75,92.0,10.0,15,0,0,-101.0,22.8,31.6,60.0,-1.3,0.0,5.2,58.1,25.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,27,-86,27.0,-83.0,44.292372,1.653082e+06,False
6,KATE,851121,6,100,27.5,-86.6,961.0,AL131985,-15.0,-35.0,-55.0,-65.0,-5.0,12.8,110,69,88.0,12.9,17,0,236,-94.0,23.6,34.8,108.0,3.0,0.0,10.8,56.3,25.3,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-87,29.0,-86.0,0.239075,1.364132e+04,False
7,KATE,851121,12,95,28.3,-86.5,965.0,AL131985,-15.0,-45.0,-55.0,-60.0,-10.0,15.9,93,69,91.0,12.1,18,0,553,-100.0,24.6,37.1,97.0,9.3,0.0,14.6,55.8,24.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-86,29.0,-86.0,0.239075,1.364132e+04,False
8,BONN,860626,0,70,28.2,-92.9,999.0,AL021986,-5.0,-40.0,-50.0,-55.0,15.0,7.6,-16,67,45.0,17.0,-68,0,190,52.0,12.8,15.5,15.0,0.7,5.7,3.6,56.1,28.7,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,28,-93,29.0,-93.0,0.724109,3.454524e+04,False
9,BONN,860626,6,75,29.0,-93.7,995.0,AL021986,-40.0,-50.0,-55.0,-65.0,10.0,15.2,-36,65,39.0,19.0,-71,0,250,54.0,20.1,23.5,-17.0,1.5,3.8,3.2,55.1,28.6,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,29,-94,29.0,-94.0,2.691408,1.015603e+05,False


### 1.8 weighted avg GDP

In [5]:
# created by Neel
fname='https://raw.githubusercontent.com/akshaym2497/RMDS_Hurricane_Forecasting/master/data/US_Hurricane_Location_GDP.csv'

Loc_GDP_2 = pd.read_csv(fname)
Loc_GDP_2=Loc_GDP_2.rename(columns={'lat':'LAT','lon':'LON','GDP':'GDP2'})
Loc_GDP_2=Loc_GDP_2[['LAT','LON','Year','City','GDP2']]
Loc_GDP_2.head()

,LAT,LON,Year,City,GDP2
0,33.92156,-78.02027,1990,Southport,10.73
1,33.92156,-78.02027,1991,Southport,11.02
2,33.92156,-78.02027,1992,Southport,11.31
3,33.92156,-78.02027,1993,Southport,11.61
4,33.92156,-78.02027,1994,Southport,11.92


##### Note: cannot match ships dataset's lat&lon

In [6]:
from pandas import Series,DataFrame
avgGDP=Merge5.groupby(['KEY']).GDPMER.mean()
avgGDP = DataFrame(avgGDP)
avgGDP=avgGDP.rename(columns={'GDPMER':'avg_GDPMER'})
avgGDP.head()

,avg_GDPMER
KEY,
ALBE-1994-07,1.844737
ALEX-2004-08,0.997174
ALEX-2010-06,0.477250
ALIC-1983-08,6.817196
ANDR-1992-08,17.221649


In [7]:
# combine avgGDP and Merge5
Merge5 = pd.merge(Merge5,avgGDP,how='left',on=['KEY'])
Merge5

,NAME,DATE_x,HOUR,VMAX,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,RHCN,TPW,PC2,SHRD2,SHRG,DIVC,U200,EPSS,ENSS,TPWC,RSST,MONTH_x,YEAR_x,DAY,TAR,KEY,storm name,landfall date,damage ranke,current damage($ 2020),base damage,State,category,winds(MPH),MONTH_y,DATE_y,Year,MONTHINT,intDay,intDate,GDP,population,LatInt,LonInt,grid_Lat,grid_Lon,GDPMER,popgpw,is_land,avg_GDPMER
0,ALIC,830818,0,95,28.4,-94.8,969.0,AL031983,-15.0,-60.0,-70.0,-75.0,20.0,2.0,15,60,58.0,11.2,-46,0,83,13.0,5.8,14.3,23.0,-3.1,7.0,5.5,60.5,29.8,8,1983,18,0,ALIC-1983-08,Alicia,"Aug 18,1983",26,2.233000e+10,3.000000e+09,TX,3,115,Aug,18,1983,8,18,18,NaN,NaN,28,-95,29.0,-95.0,6.817196,2.341434e+05,False,6.817196
1,DIAN,840913,0,85,33.8,-77.4,972.0,AL101984,-20.0,-40.0,-40.0,-30.0,-10.0,1.7,-41,53,87.0,10.6,-37,0,465,-23.0,5.4,11.0,-61.0,1.0,0.9,4.0,58.2,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-77,34.0,-77.0,1.767393,7.252184e+04,False,4.748127
2,DIAN,840913,12,65,34.0,-78.3,990.0,AL101984,-20.0,-20.0,-10.0,-5.0,-20.0,5.2,-21,53,60.0,12.9,-41,0,741,-15.0,8.3,12.0,-44.0,-0.1,0.0,7.1,56.4,26.7,9,1984,13,0,DIAN-1984-09,Diana,"Sep 13,1984",144,6.200000e+08,6.500000e+07,NC,2,110,Sep,13,1984,9,13,13,NaN,NaN,34,-78,34.0,-78.0,7.728861,3.242476e+05,True,4.748127
3,ELEN,850902,0,110,29.4,-85.9,953.0,AL051985,-10.0,-65.0,-85.0,-90.0,5.0,9.9,13,61,76.0,12.4,-45,0,212,-21.0,9.9,17.9,12.0,-5.5,3.4,3.3,61.1,29.4,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,29,-86,29.0,-86.0,0.239075,1.364132e+04,False,5.722243
4,ELEN,850902,12,100,30.2,-88.8,959.0,AL051985,-55.0,-75.0,-80.0,-80.0,-10.0,11.9,2,59,75.0,12.0,-59,0,324,-19.0,13.1,22.1,-7.0,-5.0,0.1,4.7,61.5,29.3,9,1985,2,0,ELEN-1985-09,Elena,"Sep 02,1985",67,5.360000e+09,1.250000e+09,MS,3,115,Sep,2,1985,9,2,2,NaN,NaN,30,-89,30.0,-89.0,11.205411,5.601347e+05,False,5.722243
5,KATE,851121,0,105,26.8,-86.5,954.0,AL131985,-10.0,-25.0,-55.0,-65.0,0.0,18.7,66,75,92.0,10.0,15,0,0,-101.0,22.8,31.6,60.0,-1.3,0.0,5.2,58.1,25.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,27,-86,27.0,-83.0,44.292372,1.653082e+06,False,14.923507
6,KATE,851121,6,100,27.5,-86.6,961.0,AL131985,-15.0,-35.0,-55.0,-65.0,-5.0,12.8,110,69,88.0,12.9,17,0,236,-94.0,23.6,34.8,108.0,3.0,0.0,10.8,56.3,25.3,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-87,29.0,-86.0,0.239075,1.364132e+04,False,14.923507
7,KATE,851121,12,95,28.3,-86.5,965.0,AL131985,-15.0,-45.0,-55.0,-60.0,-10.0,15.9,93,69,91.0,12.1,18,0,553,-100.0,24.6,37.1,97.0,9.3,0.0,14.6,55.8,24.7,11,1985,21,0,KATE-1985-11,Kate,"Nov 21,1985",108,1.800000e+09,3.000000e+08,FL,2,100,Nov,21,1985,11,21,21,NaN,NaN,28,-86,29.0,-86.0,0.239075,1.364132e+04,False,14.923507
8,BONN,860626,0,70,28.2,-92.9,999.0,AL021986,-5.0,-40.0,-50.0,-55.0,15.0,7.6,-16,67,45.0,17.0,-68,0,190,52.0,12.8,15.5,15.0,0.7,5.7,3.6,56.1,28.7,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,28,-93,29.0,-93.0,0.724109,3.454524e+04,False,1.707759
9,BONN,860626,6,75,29.0,-93.7,995.0,AL021986,-40.0,-50.0,-55.0,-65.0,10.0,15.2,-36,65,39.0,19.0,-71,0,250,54.0,20.1,23.5,-17.0,1.5,3.8,3.2,55.1,28.6,6,1986,26,0,BONN-1986-06,Bonnie,"Jun 26,1986",239,7.000000e+06,2.000000e+06,TX,1,85,Jun,26,1986,6,26,26,NaN,NaN,29,-94,29.0,-94.0,2.691408,1.015603e+05,False,1.707759


### Part 2: Forecasting Model

In [ ]:
# Variable names for predictors
#PredictorName=['PER','SHRD','D200','TPW','PC2','SDBT','POT','OHC','VMX0','prcp_surp','GDP','population']

# Data within training and validating years
#data_train = Merge4[(Merge4['YEAR_x']>=year_train[0]) & (Merge4['YEAR_x']<=year_train[1])]

# All predictors for training and validating
#XData = data_train[PredictorName]

# All predictand for training and validating
#YData = data_train['current damage($ 2020)']

In [ ]:
# split the training dataset and testing dataset randomly
from sklearn.model_selection import train_test_split
Merge5_train, Merge5_test = train_test_split(Merge5, test_size=0.3,random_state=123) # 70% training dataset, 30% testing dataset
# random_state is the same as set.seed()

In [ ]:
# no 'VMX0' and 'OHC' in the new ships dataset (from 1982)
PredictorName=['PER','SHRD','D200','TPW','PC2','SDBT','POT','GDPMER','popgpw']
# All predictors for training and validating
XData = Merge5_train[PredictorName]

# All predictand for training and validating
YData = Merge5_train['current damage($ 2020)']

In [ ]:
#Merge5_train, Merge5_test
# All predictors for forecast
XData_fcst = Merge5_test[PredictorName]
#XData_fcst

# All truth of predictand for forecast
YData_fcst = Merge5_test['current damage($ 2020)']
#YData_fcst

### For K-fold

In [8]:
# no 'VMX0' and 'OHC' in the new ships dataset (from 1982)
PredictorName=['PER','SHRD','D200','TPW','PC2','SDBT','POT','avg_GDPMER','popgpw']
X=Merge5[PredictorName]
y=Merge5['current damage($ 2020)']

In [9]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=12344, shuffle=True)
for train_index, test_index in kf.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   3   4   5   6   7   8   9  10  11  12  14  15  16  17  18  19
  20  22  23  24  25  28  29  30  31  32  33  35  36  38  39  40  41  43
  44  45  46  47  48  50  51  54  55  56  57  58  59  60  61  63  64  66
  68  69  70  71  72  73  75  76  77  78  80  81  82  83  84  85  86  87
  88  89  90  91  92  93  95  96  99 100 102 103 104 105 106 107 108 110
 112 113 116 117 118 119 121 122 124 125 126 128 129 130 131 132 133 134] TEST: [  2  13  21  26  27  34  37  42  49  52  53  62  65  67  74  79  94  97
  98 101 109 111 114 115 120 123 127 135]
TRAIN: [  0   1   2   4   5   6   7   9  11  12  13  14  15  17  19  20  21  22
  23  26  27  28  29  30  31  32  33  34  35  37  38  40  41  42  43  45
  47  49  50  51  52  53  54  55  56  57  58  59  61  62  64  65  66  67
  69  70  71  73  74  75  76  77  78  79  80  81  82  83  84  85  86  89
  90  91  93  94  95  96  97  98  99 101 102 104 106 107 109 110 111 112
 114 115 116 118 119 120 121 122 123 124 125 127 128 129 130 

### 2.1 Random Forest (re-assign)

### Before, we split training and testing dataset using 'Years', while the climate patterns may change during 10 years. Now, I tried spliting training and testing dataset randomly.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
regressor = RandomForestRegressor(n_estimators=100)
regressor.fit(XData, YData)
train_y_pred = regressor.predict(XData)
# validation dataset R^2
score=cross_val_score(regressor, XData, YData, cv=10,scoring='r2')
score 

In [ ]:
# train R^2
from sklearn.metrics import r2_score
RF_r2=r2_score(YData , train_y_pred)
RF_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData)), YData, 'r', label='Y train')
plt.plot(range(len(YData)), train_y_pred, 'b', label='Y pred')
plt.legend()

In [ ]:
# Predicted values for x
y_pred_fcst = regressor.predict(XData_fcst)
#y_pred_fcst

In [ ]:
# forecast MSE and RMSE
from sklearn import metrics
RF_MSE = metrics.mean_squared_error(YData_fcst, y_pred_fcst)
RMSE = np.sqrt(metrics.mean_squared_error(YData_fcst, y_pred_fcst))
print('MSE:',RF_MSE,'RMSE:',RMSE)

In [ ]:
# forecast R^2
from sklearn.metrics import r2_score
fore_RF_r2=r2_score(YData_fcst , y_pred_fcst)
fore_RF_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData_fcst)), YData_fcst, 'r', label='Y test')
plt.plot(range(len(YData_fcst)), y_pred_fcst, 'b', label='Y pred')
plt.legend()

##### Training R^2 is still better than Forecasting  R^2, there may be the problem of overfitting.

### 2.1.1 K-fold Random Forest

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn import metrics


RF_r2=[]
RF_MSE=[]
RF_RMSE=[]
fore_RF_r2=[]
for train_index, test_index in kf.split(X, y):
    XTrain=X.iloc[train_index]
    yTrain=y.iloc[train_index]
    XTest=X.iloc[test_index]
    yTest=y.iloc[test_index]
    # Train
    regressor = RandomForestRegressor(n_estimators=100)
    regressor.fit(XTrain, yTrain)
    train_y_pred = regressor.predict(XTrain)
    RF_r2.append(r2_score(yTrain , train_y_pred))
    # Forecast
    y_pred_fcst = regressor.predict(XTest)
    RF_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
    RF_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
    fore_RF_r2.append(r2_score(yTest , y_pred_fcst))
    

mean_RF_r2=np.mean(RF_r2)
mean_RF_MSE=np.mean(RF_MSE)
mean_RF_RMSE=np.mean(RF_RMSE)
mean_RF_fore_r2=np.mean(fore_RF_r2)
print('mean_RF_r2:',mean_RF_r2,'mean_RF_MSE:',mean_RF_MSE,'mean_RF_RMSE:',mean_RF_RMSE,'mean_RF_fore_r2:',mean_RF_fore_r2)

mean_RF_r2: 0.8915774110559885 mean_RF_MSE: 4.0276458332083873e+20 mean_RF_RMSE: 18502028687.16665 mean_RF_fore_r2: 0.2768881663199652


### 2.2 Linear Regression(re-assign)

In [ ]:
from sklearn.linear_model import LinearRegression 
linreg = LinearRegression()
linreg.fit(XData, YData)
train_y_pred = linreg.predict(XData)
# validation dataset R^2
score=cross_val_score(regressor, XData, YData, cv=10,scoring='r2')
score 

In [ ]:
# train R^2
from sklearn.metrics import r2_score
LR_r2=r2_score(YData , train_y_pred)
LR_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData)), YData, 'r', label='Y train')
plt.plot(range(len(YData)), train_y_pred, 'b', label='Y pred')
plt.legend()

In [ ]:
# Predicted values for x
y_pred_fcst = linreg.predict(XData_fcst)
#y_pred_fcst

In [ ]:
# forecast MSE, RMSE
from sklearn import metrics
LR_MSE = metrics.mean_squared_error(YData_fcst, y_pred_fcst)
RMSE = np.sqrt(metrics.mean_squared_error(YData_fcst, y_pred_fcst))
print('MSE:',LR_MSE,'RMSE:',RMSE)

In [ ]:
# forecast R^2
from sklearn.metrics import r2_score
fore_LR_r2=r2_score(YData_fcst , y_pred_fcst)
fore_LR_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData_fcst)), YData_fcst, 'r', label='Y test')
plt.plot(range(len(YData_fcst)), y_pred_fcst, 'b', label='Y pred')
plt.legend()

### 2.2.2 K-fold Linear Reg

In [11]:
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import r2_score
from sklearn import metrics

LR_r2=[]
LR_MSE=[]
LR_RMSE=[]
fore_LR_r2=[]
for train_index, test_index in kf.split(X, y):
    XTrain=X.iloc[train_index]
    yTrain=y.iloc[train_index]
    XTest=X.iloc[test_index]
    yTest=y.iloc[test_index]
    # Train
    regressor = LinearRegression()
    regressor.fit(XTrain, yTrain)
    train_y_pred = regressor.predict(XTrain)
    LR_r2.append(r2_score(yTrain , train_y_pred))
    # Forecast
    y_pred_fcst = regressor.predict(XTest)
    LR_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
    LR_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
    fore_LR_r2.append(r2_score(yTest , y_pred_fcst))
    

mean_LR_r2=np.mean(LR_r2)
mean_LR_MSE=np.mean(LR_MSE)
mean_LR_RMSE=np.mean(LR_RMSE)
mean_LR_fore_r2=np.mean(fore_LR_r2)
print('mean_LR_r2:',mean_LR_r2,'mean_LR_MSE:',mean_LR_MSE,'mean_LR_RMSE:',mean_LR_RMSE,'mean_LR_fore_r2:',mean_LR_fore_r2)

mean_LR_r2: 0.1485357924763647 mean_LR_MSE: 6.522217381337287e+20 mean_LR_RMSE: 24235865462.60997 mean_LR_fore_r2: -0.09640851763981799


### 2.3 Gradient Boosting Regressor

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0, loss='ls')
est.fit(XData,YData)
est_train_pred_y=est.predict(XData)
mean_squared_error(YData,est_train_pred_y)   

In [ ]:
# validation dataset R^2
score=cross_val_score(regressor, XData, YData, cv=10,scoring='r2')

# train R^2
from sklearn.metrics import r2_score
GB_r2=r2_score(YData , est_train_pred_y)

print(score,GB_r2)

In [ ]:
import statistics
statistics.mean(score)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData)), YData, 'r', label='Y train')
plt.plot(range(len(YData)), est_train_pred_y, 'b', label='Y pred')
plt.legend()

In [ ]:
# Predicted values for x
y_pred_fcst = est.predict(XData_fcst)
#y_pred_fcst

In [ ]:
# forecast MSE, RMSE
from sklearn import metrics
GB_MSE = metrics.mean_squared_error(YData_fcst, y_pred_fcst)
RMSE = np.sqrt(metrics.mean_squared_error(YData_fcst, y_pred_fcst))
print('MSE:',GB_MSE,'RMSE:',RMSE)

In [ ]:
# forecast R^2
from sklearn.metrics import r2_score
fore_GB_r2=r2_score(YData_fcst , y_pred_fcst)
fore_GB_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData_fcst)), YData_fcst, 'r', label='Y test')
plt.plot(range(len(YData_fcst)), y_pred_fcst, 'b', label='Y pred')
plt.legend()

In [ ]:
# find the best learning_rate
from sklearn.metrics import r2_score
for i in np.arange(0.1,1.1,0.1): 
    est = GradientBoostingRegressor(n_estimators=100, learning_rate=i,max_depth=1, random_state=0, loss='ls')
    est.fit(XData,YData)
    # val
    score=cross_val_score(regressor, XData, YData, cv=10,scoring='r2')
    mean_score=statistics.mean(score)
    # train
    est_train_pred_y=est.predict(XData)
    # train R^2
    r2_train=r2_score(YData,est_train_pred_y)
    # forecast
    y_pred_fcst = est.predict(XData_fcst)
    # forecast R^2
    r2_forecast_score=r2_score(YData_fcst , y_pred_fcst)
    print('i:',i,'val_R^2:',mean_score,'train R^2:',r2_train,'test R^2:',r2_forecast_score)
# then, we will choose from sklearn.metrics import r2_score =1.0

### 2.3.2 K-fold Gradient Boosting Reg

In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn import metrics

for i in np.arange(0.1,1.1,0.1):
    regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=i,max_depth=1, random_state=0, loss='ls')
    GB_r2=[]
    GB_MSE=[]
    GB_RMSE=[]
    fore_GB_r2=[]
    for train_index, test_index in kf.split(X, y):
        XTrain=X.iloc[train_index]
        yTrain=y.iloc[train_index]
        XTest=X.iloc[test_index]
        yTest=y.iloc[test_index]
        # Train
        regressor.fit(XTrain, yTrain)
        train_y_pred = regressor.predict(XTrain)
        GB_r2.append(r2_score(yTrain , train_y_pred))
        # Forecast
        y_pred_fcst = regressor.predict(XTest)
        GB_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
        GB_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
        fore_GB_r2.append(r2_score(yTest , y_pred_fcst))
    mean_GB_r2=np.mean(GB_r2)
    mean_GB_MSE=np.mean(GB_MSE)
    mean_GB_RMSE=np.mean(GB_RMSE)
    mean_GB_fore_r2=np.mean(fore_GB_r2)
    print('i:',i,'mean_GB_r2:',mean_GB_r2,'mean_GB_MSE:',mean_GB_MSE,'mean_GB_RMSE:',mean_GB_RMSE,'mean_GB_fore_r2:',mean_GB_fore_r2)
# Then, I will set learning_rate=0.8

i: 0.1 mean_GB_r2: 0.6503041874930569 mean_GB_MSE: 4.37561722890656e+20 mean_GB_RMSE: 19446895506.29706 mean_GB_fore_r2: 0.2638473744678258
i: 0.2 mean_GB_r2: 0.8125556947264438 mean_GB_MSE: 3.268347674164706e+20 mean_GB_RMSE: 16884729925.316544 mean_GB_fore_r2: 0.3723827938286294
i: 0.30000000000000004 mean_GB_r2: 0.8863013604442763 mean_GB_MSE: 2.6833090855779808e+20 mean_GB_RMSE: 15271895879.591345 mean_GB_fore_r2: 0.4255753269099406
i: 0.4 mean_GB_r2: 0.9213972703456168 mean_GB_MSE: 2.3965266848934098e+20 mean_GB_RMSE: 14221840237.03236 mean_GB_fore_r2: 0.5076054603250506
i: 0.5 mean_GB_r2: 0.9429003193099874 mean_GB_MSE: 2.4118236494117978e+20 mean_GB_RMSE: 14222644322.151459 mean_GB_fore_r2: 0.4794808953633055
i: 0.6 mean_GB_r2: 0.9551177685889456 mean_GB_MSE: 2.5434016522505264e+20 mean_GB_RMSE: 14766263110.468414 mean_GB_fore_r2: 0.34774177008718016
i: 0.7000000000000001 mean_GB_r2: 0.96669440182669 mean_GB_MSE: 2.3022221736656968e+20 mean_GB_RMSE: 13836657910.163675 mean_GB_fo

In [13]:
regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.8,max_depth=1, random_state=0, loss='ls')
GB_r2=[]
GB_MSE=[]
GB_RMSE=[]
fore_GB_r2=[]
for train_index, test_index in kf.split(X, y):
    XTrain=X.iloc[train_index]
    yTrain=y.iloc[train_index]
    XTest=X.iloc[test_index]
    yTest=y.iloc[test_index]
    # Train
    regressor.fit(XTrain, yTrain)
    train_y_pred = regressor.predict(XTrain)
    GB_r2.append(r2_score(yTrain , train_y_pred))
    # Forecast
    y_pred_fcst = regressor.predict(XTest)
    GB_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
    GB_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
    fore_GB_r2.append(r2_score(yTest , y_pred_fcst))
mean_GB_r2=np.mean(GB_r2)
mean_GB_MSE=np.mean(GB_MSE)
mean_GB_RMSE=np.mean(GB_RMSE)
mean_GB_fore_r2=np.mean(fore_GB_r2)
print('i:',i,'mean_GB_r2:',mean_GB_r2,'mean_GB_MSE:',mean_GB_MSE,'mean_GB_RMSE:',mean_GB_RMSE,'mean_GB_fore_r2:',mean_GB_fore_r2)
# Then, I will set learning_rate=0.8

i: 1.0 mean_GB_r2: 0.9754790451817208 mean_GB_MSE: 2.016048929708812e+20 mean_GB_RMSE: 13250016929.316288 mean_GB_fore_r2: 0.5353338048819568


### 2.4 AdaBoostRegressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adaReg = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=1.0)
adaReg.fit(XData,YData)
ada_train_pred_y=adaReg.predict(XData)

In [ ]:
# validation dataset R^2
score=cross_val_score(adaReg, XData, YData, cv=10,scoring='r2')

# train R^2
from sklearn.metrics import r2_score
AB_r2=r2_score(YData , ada_train_pred_y)

print(score,AB_r2)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData)), YData, 'r', label='Y train')
plt.plot(range(len(YData)), ada_train_pred_y, 'b', label='Y pred')
plt.legend()

In [ ]:
# Predicted values for x
y_pred_fcst = adaReg.predict(XData_fcst)
#y_pred_fcst

In [ ]:
# forecast MSE, RMSE
from sklearn import metrics
AB_MSE = metrics.mean_squared_error(YData_fcst, y_pred_fcst)
RMSE = np.sqrt(metrics.mean_squared_error(YData_fcst, y_pred_fcst))
print('MSE:',AB_MSE,'RMSE:',RMSE)

In [ ]:
# forecast R^2
from sklearn.metrics import r2_score
fore_AB_r2=r2_score(YData_fcst , y_pred_fcst)
fore_AB_r2

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(len(YData_fcst)), YData_fcst, 'r', label='Y test')
plt.plot(range(len(YData_fcst)), y_pred_fcst, 'b', label='Y pred')
plt.legend()

In [ ]:
# find the best learning_rate
from sklearn.metrics import r2_score
for i in np.arange(0.1,1.1,0.1): 
    adaReg = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=i)
    adaReg.fit(XData,YData)
    # train
    ada_train_pred_y=adaReg.predict(XData)
    # train R^2
    r2_train=r2_score(YData,ada_train_pred_y)
    # forecast
    y_pred_fcst = adaReg.predict(XData_fcst)
    # forecast R^2
    r2_forecast_score=r2_score(YData_fcst , y_pred_fcst)
    print('i:',i,'train R^2:',r2_train,'test R^2:',r2_forecast_score)
# then, we will choose from sklearn.metrics import r2_score =1.0

### 2.4.2 K-fold Ada Boosting Reg

In [14]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score
from sklearn import metrics

for i in np.arange(0.1,1.1,0.1):
    regressor = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=i)
    AB_r2=[]
    AB_MSE=[]
    AB_RMSE=[]
    fore_AB_r2=[]
    for train_index, test_index in kf.split(X, y):
        XTrain=X.iloc[train_index]
        yTrain=y.iloc[train_index]
        XTest=X.iloc[test_index]
        yTest=y.iloc[test_index]
        # Train
        #regressor = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=1.0)
        regressor.fit(XTrain, yTrain)
        train_y_pred = regressor.predict(XTrain)
        AB_r2.append(r2_score(yTrain , train_y_pred))
        # Forecast
        y_pred_fcst = regressor.predict(XTest)
        AB_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
        AB_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
        fore_AB_r2.append(r2_score(yTest , y_pred_fcst))
    mean_AB_r2=np.mean(AB_r2)
    mean_AB_MSE=np.mean(AB_MSE)
    mean_AB_RMSE=np.mean(AB_RMSE)
    mean_AB_fore_r2=np.mean(fore_AB_r2)
    print('i:',i,'mean_AB_r2:',mean_AB_r2,'mean_AB_MSE:',mean_AB_MSE,'mean_AB_RMSE:',mean_AB_RMSE,'mean_AB_fore_r2:',mean_AB_fore_r2)
# Then, I will set learning_rate=0.8

i: 0.1 mean_AB_r2: 0.9642036245103004 mean_AB_MSE: 3.399180791507809e+20 mean_AB_RMSE: 15718097066.658081 mean_AB_fore_r2: 0.3597968263122118
i: 0.2 mean_AB_r2: 0.9612861763138586 mean_AB_MSE: 3.5023244600510185e+20 mean_AB_RMSE: 15965394120.779123 mean_AB_fore_r2: 0.3572448760017543
i: 0.30000000000000004 mean_AB_r2: 0.9578364975184825 mean_AB_MSE: 3.5514391727741344e+20 mean_AB_RMSE: 16110032517.157227 mean_AB_fore_r2: 0.338277237235595
i: 0.4 mean_AB_r2: 0.9571634107366126 mean_AB_MSE: 3.472349576603056e+20 mean_AB_RMSE: 16169835954.595713 mean_AB_fore_r2: 0.33777279302721713
i: 0.5 mean_AB_r2: 0.9560701299483266 mean_AB_MSE: 3.506578152515381e+20 mean_AB_RMSE: 16188809486.792332 mean_AB_fore_r2: 0.33351381211760395
i: 0.6 mean_AB_r2: 0.9587949953841839 mean_AB_MSE: 3.597949179493667e+20 mean_AB_RMSE: 16396745002.534698 mean_AB_fore_r2: 0.30874233433617293
i: 0.7000000000000001 mean_AB_r2: 0.9588977693878302 mean_AB_MSE: 3.562503466607014e+20 mean_AB_RMSE: 16361433535.474575 mean_AB

In [15]:

regressor = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=0.8)
AB_r2=[]
AB_MSE=[]
AB_RMSE=[]
fore_AB_r2=[]
for train_index, test_index in kf.split(X, y):
    XTrain=X.iloc[train_index]
    yTrain=y.iloc[train_index]
    XTest=X.iloc[test_index]
    yTest=y.iloc[test_index]
    # Train
    #regressor = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=1.0)
    regressor.fit(XTrain, yTrain)
    train_y_pred = regressor.predict(XTrain)
    AB_r2.append(r2_score(yTrain , train_y_pred))
    # Forecast
    y_pred_fcst = regressor.predict(XTest)
    AB_MSE.append(metrics.mean_squared_error(yTest, y_pred_fcst))
    AB_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, y_pred_fcst)))
    fore_AB_r2.append(r2_score(yTest , y_pred_fcst))
mean_AB_r2=np.mean(AB_r2)
mean_AB_MSE=np.mean(AB_MSE)
mean_AB_RMSE=np.mean(AB_RMSE)
mean_AB_fore_r2=np.mean(fore_AB_r2)
print('i:',i,'mean_AB_r2:',mean_AB_r2,'mean_AB_MSE:',mean_AB_MSE,'mean_AB_RMSE:',mean_AB_RMSE,'mean_AB_fore_r2:',mean_AB_fore_r2)

i: 1.0 mean_AB_r2: 0.9602239964110394 mean_AB_MSE: 3.359340810832098e+20 mean_AB_RMSE: 15775944857.03407 mean_AB_fore_r2: 0.37066998553780495


### Part 3: Weighted Avg

In [ ]:
# simply weighted avg these methods predicted values
# RF regressor
RF_y_pred_fcst = regressor.predict(XData_fcst)
# linear reg
LR_y_pred_fcst = linreg.predict(XData_fcst)
# GDBT
GD_y_pred_fcst = est.predict(XData_fcst)
# ada reg
ada_y_pred_fcst = adaReg.predict(XData_fcst)
n=XData_fcst.shape[0]
weights=[0.2,0.1,0.2,0.5]
y_pred_fcst=[]
for i in range(n):
    y_pred_fcst.append(weights[0]*RF_y_pred_fcst[i]+weights[1]*LR_y_pred_fcst[i]+weights[2]*GD_y_pred_fcst[i]+weights[3]*ada_y_pred_fcst[i])

In [ ]:
y_pred_fcst2 = pd.Series(y_pred_fcst)

In [ ]:
# forecast MSE, RMSE
from sklearn import metrics
avg_MSE = metrics.mean_squared_error(YData_fcst, y_pred_fcst2)
RMSE = np.sqrt(metrics.mean_squared_error(YData_fcst, y_pred_fcst2))
print('MSE:',avg_MSE,'RMSE:',RMSE)

In [ ]:
# forecast R^2
from sklearn.metrics import r2_score
fore_avg_r2=r2_score(YData_fcst , y_pred_fcst2)
fore_avg_r2

In [ ]:
# forecast
plt.figure(figsize=(15,5))
plt.plot(range(len(YData_fcst)), YData_fcst, 'r', label='Y test')
plt.plot(range(len(YData_fcst)), y_pred_fcst, 'b', label='Y pred')
plt.legend()

### 3.1 K-fold weighted avg

In [16]:
avg_r2=[]
avg_MSE=[]
avg_RMSE=[]
fore_avg_r2=[]
weights=[0.2,0.1,0.2,0.5]
for train_index, test_index in kf.split(X, y):
    XTrain=X.iloc[train_index]
    yTrain=y.iloc[train_index]
    XTest=X.iloc[test_index]
    yTest=y.iloc[test_index]
    # Train
    RFreg = RandomForestRegressor(n_estimators=100)
    LRreg =LinearRegression()
    GBreg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.8,max_depth=1, random_state=0, loss='ls')
    ABreg = AdaBoostRegressor(random_state=0, n_estimators=100,learning_rate=0.8)
    
    RFreg.fit(XTrain, yTrain)
    LRreg.fit(XTrain, yTrain)
    GBreg.fit(XTrain, yTrain)
    ABreg.fit(XTrain, yTrain)
    
    RF_train_y_pred = RFreg.predict(XTrain)
    LR_train_y_pred = LRreg.predict(XTrain)
    GB_train_y_pred = GBreg.predict(XTrain)
    AB_train_y_pred = ABreg.predict(XTrain)
    
    avg_train_y_pred=weights[0]*RF_train_y_pred+weights[1]*LR_train_y_pred+weights[2]*GB_train_y_pred+weights[3]*AB_train_y_pred
    
    avg_r2.append(r2_score(yTrain , avg_train_y_pred))
    
    # Forecast
    RF_y_pred_fcst = RFreg.predict(XTest)
    LR_y_pred_fcst = LRreg.predict(XTest)
    GB_y_pred_fcst = GBreg.predict(XTest)
    AB_y_pred_fcst = ABreg.predict(XTest)
    
    avg_y_pred_fcst=weights[0]*RF_y_pred_fcst+weights[1]*LR_y_pred_fcst+weights[2]*GB_y_pred_fcst+weights[3]*AB_y_pred_fcst
    
    avg_MSE.append(metrics.mean_squared_error(yTest, avg_y_pred_fcst))
    avg_RMSE.append(np.sqrt(metrics.mean_squared_error(yTest, avg_y_pred_fcst)))
    fore_avg_r2.append(r2_score(yTest , avg_y_pred_fcst))
mean_avg_r2=np.mean(avg_r2)
mean_avg_MSE=np.mean(avg_MSE)
mean_avg_RMSE=np.mean(avg_RMSE)
mean_avg_fore_r2=np.mean(fore_avg_r2)
print('mean_avg_r2:',mean_avg_r2,'mean_avg_MSE:',mean_avg_MSE,'mean_avg_RMSE:',mean_avg_RMSE,'mean_avg_fore_r2:',mean_avg_fore_r2)

mean_avg_r2: 0.9495676219210463 mean_avg_MSE: 2.8849231798974844e+20 mean_avg_RMSE: 15324944424.014185 mean_avg_fore_r2: 0.4536613890259348


### Part 4: Results

In [17]:
from pandas import Series,DataFrame
import pandas as pd
results={ 'Method Name':['RandomForestRegressor','Linear Regrssion','Gradient Boosting Regressor','Ada Boost Regressor','Weigted Avg'],
         'train_R^2':[mean_RF_r2,mean_LR_r2,mean_GB_r2,mean_AB_r2,mean_avg_r2],
         'forecast_R^2':[mean_RF_fore_r2,mean_LR_fore_r2,mean_GB_fore_r2,mean_AB_fore_r2,mean_avg_fore_r2],
         'forecasr_MSE':[mean_RF_MSE,mean_LR_MSE,mean_GB_MSE,mean_AB_MSE,mean_avg_RMSE],
         'forecasr_RMSE':[mean_RF_RMSE,mean_LR_RMSE,mean_GB_RMSE,mean_AB_MSE,mean_avg_RMSE],
}
resultsdf = DataFrame(results)
resultsdf

,Method Name,train_R^2,forecast_R^2,forecasr_MSE,forecasr_RMSE
0,RandomForestRegressor,0.891577,0.276888,4.027646e+20,1.850203e+10
1,Linear Regrssion,0.148536,-0.096409,6.522217e+20,2.423587e+10
2,Gradient Boosting Regressor,0.975479,0.535334,2.016049e+20,1.325002e+10
3,Ada Boost Regressor,0.960224,0.370670,3.359341e+20,3.359341e+20
4,Weigted Avg,0.949568,0.453661,1.532494e+10,1.532494e+10
